In [1]:
#Fuwei Huang
#CV_Classifier
#This code is to load,train,test data set on the mnist CV network

In [2]:
## Setup
import os
import cv2
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from imutils import build_montages
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot

from keras.models import model_from_json
from keras.models import load_model

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten


from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
# Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
# Invalid device or cannot modify virtual devices once initialized.
    pass

In [4]:
## Prepare the data

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
sample_size = [6000,12000,18000,24000,30000,36000,42000,48000,54000,60000]
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
training_data_dictionary = {limit :
    {
      'x_train': x_train[:limit],
      'y_train': y_train[:limit],
      'x_test': x_test,
      'y_test': y_test
    } for limit in sample_size}
#train data from 10%,20%,30%,40%,50%,60%,70%,80%,90%,100%

In [6]:
def small_VGGmodel(input_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',input_shape=input_shape))
    model.add(MaxPooling2D((3, 3),strides=1))
    model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=1))
    model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=1))
    model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=1))
    model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=1))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model
small_VGGmodel = small_VGGmodel()

def define_model(input_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model
basemodel = define_model()

## Train the basemodel
scorelist_base=[]
def train_basemodeldata(tdd_xtrain,tdd_ytrain,batch_size=128,epochs=10,model=basemodel):
    basemodel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    history=basemodel.fit(tdd_xtrain,tdd_ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    score = basemodel.evaluate(x_test,y_test,verbose=0)
    scorelist_base.append(score[1])
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    print(scorelist_base)

scorelistVGG=[]
def train_smallVGGmodeldata(tdd_xtrain,tdd_ytrain,batch_size=128,epochs=10,model=small_VGGmodel):
    small_VGGmodel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    history=small_VGGmodel.fit(tdd_xtrain,tdd_ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    score = small_VGGmodel.evaluate(x_test,y_test,verbose=0)
    scorelistVGG.append(score[1])
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    print(scorelistVGG)
    
#training_data_dictionary[60000]['y_test']
def model_evaluation(training_data_dictionary,predict_results):
    cmatrix=confusion_matrix(np.argmax(training_data_dictionary,axis=-1),predict_results)
    sensitivity=[]
    specificity=[]
    precision=[]
    recall=[]
    for i in range(10):
        tn=cmatrix.sum()-cmatrix.sum(axis=0)[i]-cmatrix.sum(axis=1)[i]+cmatrix[i][i]
        fp=cmatrix.sum(axis=0)[i]-cmatrix[i][i]
        tp=cmatrix[i][i]
        sensitivity.append(cmatrix[i][i]/(cmatrix.sum(axis=1)[i]))
        specificity.append(tn/(tn+fp))
        precision.append(tp/(tp+fp))
        recall.append(cmatrix[i][i]/cmatrix.sum(axis=1)[i])
    print('Sensitivity=',sensitivity)
    print("")
    print('Specificity=',specificity)
    print("")
    print('Precision=',precision)
    print("")
    print('Recall=',recall)

In [7]:
for i in sample_size:
    train_basemodeldata(training_data_dictionary[i]['x_train'],training_data_dictionary[i]['y_train'],batch_size=128,epochs=10,model=basemodel)

Epoch 1/10
43/43 [==============================] - 0s 11ms/step - loss: 1.0153 - accuracy: 0.7170 - val_loss: 0.2996 - val_accuracy: 0.9167
Epoch 2/10
43/43 [==============================] - 0s 9ms/step - loss: 0.2748 - accuracy: 0.9237 - val_loss: 0.2180 - val_accuracy: 0.9233
Epoch 3/10
43/43 [==============================] - 0s 9ms/step - loss: 0.1840 - accuracy: 0.9463 - val_loss: 0.1491 - val_accuracy: 0.9600
Epoch 4/10
43/43 [==============================] - 0s 9ms/step - loss: 0.1346 - accuracy: 0.9626 - val_loss: 0.1307 - val_accuracy: 0.9550
Epoch 5/10
43/43 [==============================] - 0s 9ms/step - loss: 0.0979 - accuracy: 0.9715 - val_loss: 0.1282 - val_accuracy: 0.9550
Epoch 6/10
43/43 [==============================] - 0s 9ms/step - loss: 0.0810 - accuracy: 0.9752 - val_loss: 0.1372 - val_accuracy: 0.9567
Epoch 7/10
43/43 [==============================] - 0s 9ms/step - loss: 0.0677 - accuracy: 0.9807 - val_loss: 0.0999 - val_accuracy: 0.9717
Epoch 8/10
43/43 [=

In [8]:
#Test the trained basemodel accuracy
base_predict_results = basemodel.predict_classes(x_test)
for index in range(20):
    print (base_predict_results[index],np.argmax(y_test[index], axis=-1))
#19/20 match

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
7 7
2 2
1 1
0 0
4 4
1 1
4 4
9 9
5 5
9 9
0 0
6 6
9 9
0 0
1 1
5 5
9 9
7 7
8 3
4 4


In [9]:
#Retrain the train_data function to clear the score list and retrain using the smallVGG model
for i in (6000,12000,18000,24000,30000,36000,42000,48000,54000,60000):
    train_smallVGGmodeldata(training_data_dictionary[i]['x_train'],training_data_dictionary[i]['y_train'],batch_size=128,epochs=10,model=small_VGGmodel)

Epoch 1/10
43/43 [==============================] - 3s 74ms/step - loss: 1.0682 - accuracy: 0.6454 - val_loss: 0.4536 - val_accuracy: 0.8650
Epoch 2/10
43/43 [==============================] - 3s 71ms/step - loss: 0.2481 - accuracy: 0.9244 - val_loss: 0.2085 - val_accuracy: 0.9383
Epoch 3/10
43/43 [==============================] - 3s 71ms/step - loss: 0.1731 - accuracy: 0.9467 - val_loss: 0.2334 - val_accuracy: 0.9367
Epoch 4/10
43/43 [==============================] - 3s 72ms/step - loss: 0.1057 - accuracy: 0.9685 - val_loss: 0.1881 - val_accuracy: 0.9483
Epoch 5/10
43/43 [==============================] - 3s 71ms/step - loss: 0.1124 - accuracy: 0.9665 - val_loss: 0.1800 - val_accuracy: 0.9450
Epoch 6/10
43/43 [==============================] - 3s 71ms/step - loss: 0.0788 - accuracy: 0.9756 - val_loss: 0.1153 - val_accuracy: 0.9700
Epoch 7/10
43/43 [==============================] - 3s 71ms/step - loss: 0.0603 - accuracy: 0.9817 - val_loss: 0.1340 - val_accuracy: 0.9617
Epoch 8/10
43

In [10]:
samllVGG_predict_results = small_VGGmodel.predict_classes(x_test)
for index in range(20):
    print (samllVGG_predict_results[index],np.argmax(y_test[index], axis=-1))
#20/20 correct

7 7
2 2
1 1
0 0
4 4
1 1
4 4
9 9
5 5
9 9
0 0
6 6
9 9
0 0
1 1
5 5
9 9
7 7
3 3
4 4


In [11]:
model_evaluation(training_data_dictionary[60000]['y_test'],samllVGG_predict_results)

Sensitivity= [0.9918367346938776, 0.9955947136563876, 0.9903100775193798, 0.998019801980198, 0.9938900203665988, 0.9865470852017937, 0.9864300626304802, 0.9883268482490273, 0.9928131416837782, 0.9821605550049554]

Specificity= [0.9995565410199556, 0.9994359842075579, 0.9993309545049064, 0.9979977753058954, 0.9981148813484143, 0.9993412384716732, 0.9991152399911524, 0.9985510477039679, 0.9992244626634168, 0.998998998998999]

Precision= [0.9959016393442623, 0.9955947136563876, 0.9941634241245136, 0.9824561403508771, 0.9828801611278952, 0.9932279909706546, 0.9916054564533053, 0.9873663751214772, 0.9928131416837782, 0.991]

Recall= [0.9918367346938776, 0.9955947136563876, 0.9903100775193798, 0.998019801980198, 0.9938900203665988, 0.9865470852017937, 0.9864300626304802, 0.9883268482490273, 0.9928131416837782, 0.9821605550049554]
